# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

This sample shows:
* How to create schemas and tables via the Trino / SQLAlchemy on an underlying Iceberg data volume
* Apache Iceberg ACID transaction and time travel capabilities used for data set versioning


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

Load Environment Variables

In [ ]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get("CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src"))
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [ ]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_LANDING_ENDPOINT"],
    aws_access_key_id=os.environ["S3_LANDING_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_LANDING_SECRET_KEY"],
)

In [ ]:
ingest_catalog = "osc_datacommons_iceberg_dev"
ingest_schema = "sec_dera"

In [ ]:
# telling sqlalchemy about catalog has to be done in the sqlstring url:
import trino
from sqlalchemy.engine import create_engine

sqlstring = "trino://{user}@{host}:{port}/{catalog}".format(
    user=os.environ["TRINO_USER"], host=os.environ["TRINO_HOST"], port=os.environ["TRINO_PORT"], catalog=ingest_catalog
)
sqlargs = {"auth": trino.auth.JWTAuthentication(os.environ["TRINO_PASSWD"]), "http_scheme": "https"}
engine = create_engine(sqlstring, connect_args=sqlargs)
connection = engine.connect()

Drop previous tables and schema to start with a fresh slate

In [ ]:
for ingest_table in ["sub", "num", "tag"]:
    sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
    print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())

sql = f"""
drop schema if exists {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

Enter the Pandas!

In [ ]:
import pandas as pd

Prepare GLEIF matching data

In [ ]:
gleif_file = s3_source.Object(os.environ["S3_LANDING_BUCKET"], f"mtiemann-GLEIF/DERA-matches.csv")
gleif_file.download_file(f"/tmp/dera-gleif.csv")
gleif_df = pd.read_csv(f"/tmp/dera-gleif.csv", header=0, sep=",", dtype=str, engine="c")
gleif_dict = {k: v for k, v in zip(gleif_df.name, gleif_df.LEI)}

Load the SUB and NUM tables into Trino.  READS ONLY 10 ROWS RIGHT NOW!

In [ ]:
# import osc_ingest_trino as osc

_p2smap = {
    "string": "varchar",
    "float32": "real",
    "Float32": "real",
    "float64": "double",
    "Float64": "double",
    "int32": "integer",
    "Int32": "integer",
    "int64": "bigint",
    "Int64": "bigint",
    "bool": "boolean",
    "category": "varchar",
    "datetime64[ns, UTC]": "timestamp(6)",
}


def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))


# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def create_table_schema_pairs(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("df must be a pandas DataFrame")
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0], t=e[1]) for e in pz])

In [ ]:
import re
import io
import uuid
import subprocess
from datetime import datetime

# Add a unique identifier to the data set
uid = str(uuid.uuid4())

dera_regex = re.compile(r" ?/.*$")

quarters = ["2020q1", "2020q2", "2020q3", "2020q4", "2021q1", "2021q2", "2021q3"]

# If our dataframe has more than this many rows, we need to chunk.
# ??? Should use df.info to better calculate the appropriate max_rowcount
max_rowcount = 78

last_df = None


def ingest_dera_table(qtr, tbl):
    global last_df

    src_file = s3_source.Object(os.environ["S3_LANDING_BUCKET"], f"SEC-DERA/{qtr}/{tbl}.txt")
    timestamp = src_file.last_modified.isoformat()
    src_file.download_file(f"/tmp/dera-{tbl}-{timestamp}.csv")
    df = pd.read_csv(
        f"/tmp/dera-{tbl}-{timestamp}.csv",
        header=0,
        sep="\t",
        dtype="string",
        keep_default_na=False,
        nrows=None,
        engine="c",
    )

    df["uuid"] = uid
    df["quarter"] = qtr
    df = df.convert_dtypes(
        infer_objects=False, convert_string=True, convert_integer=False, convert_boolean=False, convert_floating=False
    )
    # Print the output
    # print(df.dtypes)

    if tbl == "sub":
        df.name = df.name.map(lambda x: re.sub(dera_regex, "", x))
        df.name = df.name.astype("string")
        df["LEI"] = df.name.map(gleif_dict)
        df.LEI = df.LEI.astype("string")
        df.cik = df.cik.astype("int32")
        df.loc[df.sic == "", "sic"] = pd.NA
        df.sic = df.sic.astype("Int32")
        df.loc[df.ein == "", "ein"] = pd.NA
        df.ein = df.ein.astype("Int64")
        df.wksi = df.wksi.astype("bool")
        # df.wksi = df.wksi.astype('int32')
        df.period = pd.to_datetime(df.period, format="%Y%m%d", utc=True, errors="coerce")
        df.fy = pd.to_datetime(df.fy, format="%Y", utc=True, errors="coerce")
        df.filed = pd.to_datetime(df.filed, format="%Y%m%d", utc=True)
        df.accepted = pd.to_datetime(df.accepted, format="%Y-%m-%d %H:%M:%S", utc=True)
        df.prevrpt = df.prevrpt.astype("bool")
        df.detail = df.detail.astype("bool")
        # df.prevrpt = df.prevrpt.astype('int32')
        # df.detail = df.detail.astype('int32')
        df.nciks = df.nciks.astype("int32")

        cols = df.columns.tolist()
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tbl == "num":
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        df.ddate = pd.to_datetime(df.ddate, format="%Y%m%d", utc=True)
        df.qtrs = df.qtrs.astype("int32")
        df.value = df.value.astype("float64")
    elif tbl == "tag":
        df.custom = df.custom.astype("bool")
        df.abstract = df.abstract.astype("bool")
    print(df.dtypes)
    display(df.head())

    # Only drop table and create new one if we are starting from the beginning
    if qtr == quarters[0]:
        table_check = engine.execute(f"drop table if exists {ingest_catalog}.{ingest_schema}.{tbl}")
        for row in table_check.fetchall():
            print(row)

        columnschema = create_table_schema_pairs(df)

        tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{tbl}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['quarter']
)
"""
        print(tabledef)
        qres = engine.execute(tabledef)
        print(qres.fetchall())

    # method = 'multi' is important, default will not work
    # important to tell it about schema here, and catalog when you create the db connection above
    # index = False, unless you declared that as a column when you create the table
    # use 'append' mode since we already created the table
    last_df = df
    time1 = datetime.now()
    for i in range(0, len(df.index), max_rowcount):
        row_max = min(max_rowcount, len(df.index))
        print(f"{str(datetime.now())}: i = {i}; row_max = {row_max}, len(df.index) = {len(df.index)}")
        df.iloc[i : i + row_max].to_sql(
            tbl, con=engine, schema=ingest_schema, if_exists="append", index=False, method="multi"
        )
        time2 = datetime.now()
        tdiff = time2 - time1
        print(f"tiff = {tdiff}; ETA = {tdiff * len(df.index) / (i+row_max)}")

    sql = f"""
select * from {ingest_catalog}.{ingest_schema}.{tbl} limit 10
"""
    pd.read_sql(sql, engine)

In [ ]:
snapshot_tests = False

for qtr in quarters:
    for ingest_table in ["sub", "num", "tag"]:
        print(f"Ingesting table {ingest_table}; quarter = {qtr}; timestamp = {str(datetime.now())}")

        ingest_dera_table(qtr, ingest_table)

        if not snapshot_tests:
            continue

        sql = f"""
select snapshot_id, committed_at from {ingest_catalog}.{ingest_schema}."{ingest_table}$snapshots"
    order by committed_at desc
    limit 10
"""
        print(sql)
        qres = engine.execute(sql)
        snapshots = qres.fetchall()
        display(snapshots)

        previous_snapshot = snapshots[1][0]
        previous_snapshot

        sql = f"""
call {ingest_catalog}.system.rollback_to_snapshot('{ingest_schema}', '{ingest_table}', {previous_snapshot})
"""
        print(sql)
        qres = engine.execute(sql)
        print(qres.fetchall())

        sql = f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
        pd.read_sql(sql, engine)

        sql = f"""
call {ingest_catalog}.system.rollback_to_snapshot('{ingest_schema}', '{ingest_table}', {snapshots[0][0]})
"""
        print(sql)
        qres = engine.execute(sql)
        print(qres.fetchall())

        sql = f"""
select snapshot_id, committed_at from {ingest_catalog}.{ingest_schema}."{ingest_table}$snapshots"
    order by committed_at desc
    limit 10
"""
        qres = engine.execute(sql)
        snapshots = qres.fetchall()
        display(snapshots)

In [ ]:
last_df

Create metadata table for schema / dataset level information

In [ ]:
# declare variable names for metadata structure in Trino
meta_schema_name = "metastore_iceberg"
meta_table_name_dataset = "meta_tables_iceberg"
meta_table_name_fields = "meta_fields_iceberg"

In [ ]:
sub_meta_content = {
    "dataset_key": "SEC-DERA",
    "tablename": "sub",
    "title": "SEC DERA Disclosures: SUB",
    "description": """The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.""",
    "version": "quarterly",
    "release_date": "rolling",
    "fields": [
        {
            "adsh": "Accession Number. The 20-character string formed from the 18-digit number assigned by the SEC to each EDGAR submission.",
            "cik": "Central Index Key (CIK). Ten digit number assigned by the SEC to each registrant that submits filings.",
            "name": "Name of registrant. This corresponds to the name of the legal entity as recorded in EDGAR as of the filing date.",
            "sic": "Standard Industrial Classification (SIC). Four digit code assigned by the SEC as of the filing date, indicating the registrant’s type of business.",
            "countryba": "The ISO 3166-1 country of the registrant’s business address.",
            "stprba": "The state or province of the registrant’s business address, if field countryba is US or CA.",
            "cityba": "The city of the registrant’s business address.",
            "zipba": "The zip code of the registrant’s business address.",
            "bas1": "The first line of the street of the registrant’s business address.",
            "bas2": "The second line of the street of the registrant’s business address.",
            "baph": "The phone number of the registrant’s business address.",
            "countryma": "The ISO 3166-1 country of the registrant’s mailing address.",
            "stprma": "The state or province of the registrant’s mailing address, if field countryma is US or CA.",
            "cityma": "The city of the registrant’s mailing address.",
            "zipma": "The zip code of the registrant’s mailing address.",
            "mas1": "The first line of the street of the registrant’s mailing address.",
            "mas2": "The second line of the street of the registrant’s mailing address.",
            "countryinc": "The country of incorporation for the registrant.",
            "stprinc": "The state or province of incorporation for the registrant, if countryinc is US or CA.",
            "ein": "Employee Identification Number, 9 digit identification number assigned by the Internal Revenue Service to business entities operating in the United States.",
            "former": "Most recent former name of the registrant, if any.",
            "changed": "Date of change from the former name, if any.",
            "afs": "Filer status with the SEC at the time of submission:\n\
1-LAF=Large Accelerated,\n\
2-ACC=Accelerated,\n\
3-SRA=Smaller Reporting Accelerated,\n\
4-NON=Non-Accelerated,\n\
5-SML=Smaller Reporting Filer,\n\
NULL=not assigned.",
            "wksi": "Well Known Seasoned Issuer (WKSI). An issuer that meets specific SEC requirements at some point during a 60-day period preceding the date the issuer satisfies its obligation to update its shelf registration statement.",
            "fye": "Fiscal Year End Date, rounded to nearest month-end.",
            "form": "The submission type of the registrant’s filing.",
            "period": "Balance Sheet Date, rounded to nearest month-end.",
            "fy": "Fiscal Year Focus (as defined in EFM Ch. 6).",
            "fp": "Fiscal Period Focus (as defined in EFM Ch. 6) within Fiscal Year. The 10-Q for the 1st, 2nd and 3rd quarters would have a fiscal period focus of Q1, Q2 (or H1), and Q3 (or M9) respectively, and a 10-K would have a fiscal period focus of FY.",
            "filed": "The date of the registrant’s filing with the Commission.",
            "accepted": "The acceptance date and time of the registrant’s filing with the Commission. Filings accepted after 5:30pm EST are considered filed on the following business day.",
            "prevrpt": "Previous Report –TRUE indicates that the submission information was subsequently amended.",
            "detail": "TRUE indicates that the XBRL submission contains quantitative disclosures within the footnotes and schedules at the required detail level (e.g., each amount).",
            "instance": "The name of the submitted XBRL Instance Document (EX-101.INS) type data file. The name often begins with the company ticker symbol.",
            "nciks": "Number of Central Index Keys (CIK) of registrants (i.e., business units) included in the consolidating entity’s submitted filing.",
            "aciks": "Additional CIKs of co-registrants included in  a consolidating entity’s EDGAR submission, separated by spaces. If there are no other co-registrants (i.e., nciks=1), the value of aciks is NULL.  For a very small number of filers, the list of co-registrants is too long to fit in the field.  Where this is the case, PARTIAL will appear at the end of the list indicating that not all co-registrants’ CIKs are included in the field; users should refer to the complete submission file for all CIK information.",
        }
    ],
}

In [ ]:
num_meta_content = {
    "dataset_key": "SEC-DERA",
    "tablename": "num",
    "title": "SEC DERA Disclosures: NUM",
    "description": """The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.""",
    "version": "quarterly",
    "release_date": "rolling",
    "fields": [
        {
            "adsh": "Accession Number. The 20-character string formed from the 18-digit number assigned by the SEC to each EDGAR submission.",
            "tag": "The unique identifier (name) for a tag in a specific taxonomy release.",
            "version": "For a standard tag, an identifier for the taxonomy; otherwise the accession number where the tag was defined.",
            "coreg": "If specified, indicates a specific co-registrant, the parent company, or other entity (e.g., guarantor). NULL indicates the consolidated entity.",
            "ddate": "The end date for the data value, rounded to the nearest month end.",
            "qtrs": "The count of the number of quarters represented by the data value, rounded to the nearest whole number. “0” indicates it is a point-in-time value.",
            "uom": "The unit of measure for the value.",
            "value": "The value. This is not scaled, it is as found in the Interactive Data file, but is limited to four digits to the right of the decimal point.",
            "footnote": "The text of any superscripted footnotes on the value, as shown on the statement page, truncated to 512 characters, or if there is no footnote, then this field will be blank.",
        }
    ],
}

In [ ]:
tag_meta_content = {
    "dataset_key": "SEC-DERA",
    "tablename": "tag",
    "title": "SEC DERA Disclosures: TAG",
    "description": """The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.""",
    "version": "quarterly",
    "release_date": "rolling",
    "fields": [
        {
            "tag": "The unique identifier (name) for a tag in a specific taxonomy release.",
            "version": "For a standard tag, an identifier for the taxonomy; otherwise the accession number where the tag was defined.",
            "custom": "1 if tag is custom (version=adsh), 0 if it is standard. Note: This flag is technically redundant with the version and adsh columns.",
            "abstract": "1 if the tag is not used to represent a numeric fact.",
            "datatype": "If abstract=1, then NULL, otherwise the data type (e.g., monetary) for the tag.",
            "iord": "If abstract=1, then NULL; otherwise, “I” if the value is a point-in time, or “D” if the value is a duration.",
            "crdr": "If datatype = monetary, then the tag’s natural accounting balance (debit or credit); if not defined, then NULL.",
            "tlabel": "If a standard tag, then the label text provided by the taxonomy, otherwise the text provided by the filer. A tag which had neither would have a NULL value here.",
            "doc": "The detailed definition for the tag (truncated to 2048 characters). If a standard tag, then the text provided by the taxonomy, otherwise the text assigned by the filer. Some tags have neither, and this field is NULL.",
        }
    ],
}

Convert custom metadata content in json format into Pandas DataFrame

In [ ]:
schema_check = engine.execute(f"create schema if not exists {ingest_catalog}.{meta_schema_name}")
for row in schema_check.fetchall():
    print(row)

table_check = engine.execute(f"drop table if exists {ingest_catalog}.{meta_schema_name}.{meta_table_name_dataset}")
for row in table_check.fetchall():
    print(row)

for meta_content in [sub_meta_content, num_meta_content, tag_meta_content]:
    df_meta_table = pd.json_normalize(meta_content, max_level=0)
    df_meta_table = df_meta_table.drop("fields", axis=1)
    df_meta_table["schema"] = ingest_schema
    df_meta_table = df_meta_table.convert_dtypes()
    display(df_meta_table)
    df_meta_table.info(verbose=True)

    if meta_content == sub_meta_content:
        # We only need to create the metadata table once
        schema_meta_table = create_table_schema_pairs(df_meta_table)
        tabledef = f"""
create table if not exists {ingest_catalog}.{meta_schema_name}.{meta_table_name_dataset}(
{schema_meta_table}
) with (
    format = 'ORC',
    partitioning = array['dataset_key']
)
"""
        print(tabledef)
        qres = engine.execute(tabledef)
        print(qres.fetchall())

        meta_table_create = engine.execute(tabledef)
        for row in meta_table_create.fetchall():
            print(row)

    df_meta_table.to_sql(
        meta_table_name_dataset, con=engine, schema=meta_schema_name, if_exists="append", index=False, method="multi"
    )

    list_values_meta_table = df_meta_table.values.tolist()
    display(list_values_meta_table[0])

In [ ]:
meta_query_table = f"SELECT * FROM {ingest_catalog}.{meta_schema_name}.{meta_table_name_dataset} limit 10"
print(meta_query_table)
meta_table_query = engine.execute(meta_query_table)
for row in meta_table_query.fetchall():
    print(row)

Create metadata table for fields information

In [ ]:
table_check = engine.execute(f"drop table if exists {ingest_catalog}.{meta_schema_name}.{meta_table_name_fields}")
for row in table_check.fetchall():
    print(row)

In [ ]:
for meta_content in [sub_meta_content, num_meta_content, tag_meta_content]:
    df = pd.json_normalize(meta_content, record_path=["fields"]).T
    df["tablename"] = meta_content["tablename"]
    df["dataset_key"] = meta_content["dataset_key"]
    df = df.reset_index().convert_dtypes()
    df.rename(columns={"index": "fieldname", 0: "description"}, inplace=True)
    df_meta_fields = df
    display(df_meta_fields)
    df_meta_fields.info(verbose=True)

    if meta_content == sub_meta_content:
        # Only need to create this table once
        schema_meta_fields = create_table_schema_pairs(df_meta_fields)
        tabledef = f"""
create table if not exists {ingest_catalog}.{meta_schema_name}.{meta_table_name_fields}(
{schema_meta_fields}
) with (
    format = 'ORC',
    partitioning = array['dataset_key']
)
"""
        print(tabledef)
        qres = engine.execute(tabledef)
        print(qres.fetchall())

        meta_fields_create = engine.execute(tabledef)
        for row in meta_fields_create.fetchall():
            print(row)

    df_meta_fields.to_sql(
        meta_table_name_fields, con=engine, schema=meta_schema_name, if_exists="append", index=False, method="multi"
    )

    list_values_meta_fields = df_meta_fields.values.tolist()
    display(list_values_meta_fields[0])

In [ ]:
meta_query_fields = f"SELECT * FROM {ingest_catalog}.{meta_schema_name}.{meta_table_name_fields} limit 10"
print(meta_query_table)
meta_table_query = engine.execute(meta_query_table)
for row in meta_table_query.fetchall():
    print(row)